In [28]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import calendar

In [29]:
# La fonction renvoie sous forme de dataframe, le taux d'occupation et le débit pour chaque heure de la journée


def get_road_data_by_date(year, month, day, libelle, libelle_nd_amont, libelle_nd_aval):

    # API Url
    api_url = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/comptages-routiers-permanents/records"

    # Request parameters
    params = {
        "limit": 24,
        "order_by": "t_1h",
        "timezone": "CET",
        "refine": [
            f"t_1h:{year}/{month}/{day}",
            f"libelle_nd_amont:{libelle_nd_amont}",
            f"libelle_nd_aval:{libelle_nd_aval}",
            f"libelle:{libelle}"
        ]
    }

    # Get response
    response = requests.get(api_url, params=params)

    # Fetch data
    if response.status_code == 200:
        records = response.json().get("results", [])
        if records:
            df = pd.DataFrame(columns=["year", "month", "day", "hour", "flow rate", "occupancy rate", "trafic state"])
            for index, record in enumerate(records):
                t_1h = record["t_1h"]
                q = record["q"]
                k = record["k"]
                trafic_state = record["etat_trafic"]
                year = t_1h[:4]
                month = t_1h[5:7]
                day = t_1h[8:10]
                hour = t_1h[11:13]
                
                df.loc[index] = [year, month, day, hour, q, k, trafic_state]
        else:
            print("No data found.")
    else:
        print(f"API Error : {response.status_code}")

    return df


In [30]:
print(get_road_data_by_date("2023", "12", "06", "AV_Champs_Elysees", "Av_Champs_Elysees-Washington", "Av_Champs_Elysees-Berri"))
print(get_road_data_by_date("2023", "12", "06", "Convention", "Convention-Blomet", "Lecourbe-Convention"))
print(get_road_data_by_date("2023", "12", "06", "St_Antoine", "Bastille-St_Antoine", "St_Antoine-Jacques_Coeur"))

# q: h flow rate (number of vehicle per hour)
# k: occupancy rate (time spent by a vehicle in one hour, in %, 100% = 1 hour)

    year month day hour  flow rate  occupancy rate trafic state
0   2023    12  06   00      684.0        11.75333       Fluide
1   2023    12  06   01      624.0         8.16334       Fluide
2   2023    12  06   02      478.0         5.59167       Fluide
3   2023    12  06   03      664.0        14.25889       Fluide
4   2023    12  06   04      418.0         9.83778       Fluide
5   2023    12  06   05      261.0         3.38778       Fluide
6   2023    12  06   06      442.0         5.95889       Fluide
7   2023    12  06   07      777.0        10.27667       Fluide
8   2023    12  06   08     1083.0        11.97778       Fluide
9   2023    12  06   09     1408.0        19.71333   Pré-saturé
10  2023    12  06   10     1523.0        22.98500   Pré-saturé
11  2023    12  06   11     1400.0        23.02445   Pré-saturé
12  2023    12  06   12     1391.0        23.07111   Pré-saturé
13  2023    12  06   13     1346.0        28.98111   Pré-saturé
14  2023    12  06   14     1139.0      

In [31]:
# La fonction renvoie sous forme de dataframe, le taux d'occupation et le débit, pour un mois donné


def get_road_data_per_month(year, month, libelle, libelle_nd_amont, libelle_nd_aval):

    number_of_days = calendar.monthrange(int(year), int(month))[1]

    dfs = []

    for index in range(number_of_days):
        day = str(index + 1).zfill(2)
        dfs.append(get_road_data_by_date("2023", "12", day, libelle, libelle_nd_amont, libelle_nd_aval))
    
    df = pd.concat(dfs, ignore_index=True)
    return df


In [32]:
print(get_road_data_per_month("2024", "11", "AV_Champs_Elysees", "Av_Champs_Elysees-Washington", "Av_Champs_Elysees-Berri"))
#print(get_road_data_per_month("2024", "11", "Convention", "Convention-Blomet", "Lecourbe-Convention"))
#print(get_road_data_per_month("2024", "11", "St_Antoine", "Bastille-St_Antoine", "St_Antoine-Jacques_Coeur"))

     year month day hour  flow rate  occupancy rate trafic state
0    2023    12  01   00      863.0        13.60389       Fluide
1    2023    12  01   01      857.0        12.49333       Fluide
2    2023    12  01   02      685.0        11.09889       Fluide
3    2023    12  01   03      694.0        13.02445       Fluide
4    2023    12  01   04      636.0        14.31389       Fluide
..    ...   ...  ..  ...        ...             ...          ...
691  2023    12  30   19      953.0        41.37667       Saturé
692  2023    12  30   20      877.0        35.82500       Saturé
693  2023    12  30   21      905.0        34.69611       Saturé
694  2023    12  30   22      878.0        36.58945       Saturé
695  2023    12  30   23      633.0        28.09889   Pré-saturé

[696 rows x 7 columns]
